In [1]:

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Load and check

In [2]:
RAW_DATA_34_PATH = "data/34.csv"
data_34 = pd.read_csv(RAW_DATA_34_PATH)

In [3]:
data_34.shape

(32472, 40)

In [4]:
data_34.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [5]:
data_34

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2023-173854,2023-01-02,1,Vente,416850.0,84.0,NaN,RUE SAINTE TERESA DE CALCUTTA,1362,34170.0,34057,Castelnau-le-Lez,34,NaN,NaN,34057000BX0086,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.917974,43.653590
1,2023-173854,2023-01-02,1,Vente,416850.0,84.0,NaN,RUE SAINTE TERESA DE CALCUTTA,1362,34170.0,34057,Castelnau-le-Lez,34,NaN,NaN,34057000BX0086,NaN,NaN,9.0,86.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,Maison,85.0,4.0,NaN,NaN,NaN,NaN,NaN,3.917974,43.653590
2,2023-173855,2023-01-05,1,Vente,158730.0,101.0,NaN,AV PIERRE RACINE,0035,34280.0,34344,La Grande-Motte,34,NaN,NaN,34344000AH0046,NaN,NaN,94.0,20.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,25.0,1.0,NaN,NaN,NaN,NaN,NaN,4.086656,43.558880
3,2023-173856,2023-01-03,1,Vente,28000.0,31.0,NaN,BD JEAN MATHIEU GRANGENT,0853,34200.0,34301,Sète,34,NaN,NaN,34301000BR0190,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.666590,43.398524
4,2023-173857,2023-01-04,1,Vente,147000.0,3.0,NaN,RTE DE LAVERUNE,3200,34070.0,34172,Montpellier,34,NaN,NaN,34172000HX0494,NaN,NaN,46.0,NaN,8.0,43.95,NaN,NaN,NaN,NaN,NaN,NaN,2,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.858816,43.604228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32467,2023-188438,2023-02-21,1,Adjudication,56000.0,6.0,NaN,RUE DE LA REPUBLIQUE,6920,34500.0,34032,Béziers,34,NaN,NaN,34032000PY0085,NaN,NaN,15.0,NaN,16.0,NaN,17.0,NaN,18.0,NaN,19.0,NaN,10,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.214690,43.344708
32468,2023-188438,2023-02-21,1,Adjudication,56000.0,6.0,NaN,RUE DE LA REPUBLIQUE,6920,34500.0,34032,Béziers,34,NaN,NaN,34032000PY0085,NaN,NaN,15.0,NaN,16.0,NaN,17.0,NaN,18.0,NaN,19.0,NaN,10,2.0,Appartement,130.0,4.0,NaN,NaN,NaN,NaN,NaN,3.214690,43.344708
32469,2023-188438,2023-02-21,1,Adjudication,56000.0,6.0,NaN,RUE DE LA REPUBLIQUE,6920,34500.0,34032,Béziers,34,NaN,NaN,34032000PY0085,NaN,NaN,15.0,NaN,16.0,NaN,17.0,NaN,18.0,NaN,19.0,NaN,10,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.214690,43.344708
32470,2023-188438,2023-02-21,1,Adjudication,56000.0,6.0,NaN,RUE DE LA REPUBLIQUE,6920,34500.0,34032,Béziers,34,NaN,NaN,34032000PY0085,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.214690,43.344708


# Keep only sales

In [6]:
data_34["nature_mutation"].value_counts()

Vente                                 30228
Vente en l'état futur d'achèvement     1817
Echange                                 225
Vente terrain à bâtir                   158
Adjudication                             44
Name: nature_mutation, dtype: int64

In [7]:
mutation_mask = data_34["nature_mutation"] == "Vente"

In [8]:
mutation_mask.value_counts()

True     30228
False     2244
Name: nature_mutation, dtype: int64

In [9]:
data_34 = data_34[mutation_mask]

# Keep only id mutations with Appartments

In [10]:
data_34["type_local"].value_counts(dropna=False)

Dépendance                                  10011
NaN                                          7601
Appartement                                  6620
Maison                                       4763
Local industriel. commercial ou assimilé     1233
Name: type_local, dtype: int64

In [11]:
data_34.groupby("id_mutation")["type_local"].apply(lambda x: list(x))

id_mutation
2023-173854         [Dépendance, Maison]
2023-173855                [Appartement]
2023-173856                 [Dépendance]
2023-173857    [Dépendance, Appartement]
2023-173858    [Appartement, Dépendance]
                         ...            
2023-188430                     [Maison]
2023-188431                        [nan]
2023-188433                   [nan, nan]
2023-188434                     [Maison]
2023-188435                [Appartement]
Name: type_local, Length: 13597, dtype: object

In [12]:
mutation_only_appartments = data_34.groupby("id_mutation")["type_local"]\
    .apply(lambda x: all([local == "Appartement" for local in list(x)]))

In [13]:
mutation_only_appartments

id_mutation
2023-173854    False
2023-173855     True
2023-173856    False
2023-173857    False
2023-173858    False
               ...  
2023-188430    False
2023-188431    False
2023-188433    False
2023-188434    False
2023-188435     True
Name: type_local, Length: 13597, dtype: bool

In [14]:
mutation_only_appartments.value_counts()

False    12383
True      1214
Name: type_local, dtype: int64

In [15]:
mutation_only_appartments_ids = mutation_only_appartments[mutation_only_appartments].index

In [16]:
mutation_only_appartments_ids

Index(['2023-173855', '2023-173861', '2023-173862', '2023-173864',
       '2023-173870', '2023-173885', '2023-173894', '2023-173941',
       '2023-173946', '2023-173971',
       ...
       '2023-188360', '2023-188368', '2023-188379', '2023-188399',
       '2023-188400', '2023-188408', '2023-188409', '2023-188418',
       '2023-188419', '2023-188435'],
      dtype='object', name='id_mutation', length=1214)

In [17]:
data_34 = data_34[data_34['id_mutation'].isin(mutation_only_appartments_ids)]

In [18]:
data_34

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
2,2023-173855,2023-01-05,1,Vente,158730.0,101.0,NaN,AV PIERRE RACINE,0035,34280.0,34344,La Grande-Motte,34,NaN,NaN,34344000AH0046,NaN,NaN,94.0,20.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,25.0,1.0,NaN,NaN,NaN,NaN,NaN,4.086656,43.558880
12,2023-173861,2023-01-05,1,Vente,216000.0,10.0,A,RUE BASSE,0010,34670.0,34022,Baillargues,34,NaN,NaN,34022000AH0068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,24.0,2.0,S,sols,NaN,NaN,103.0,4.014809,43.660787
13,2023-173861,2023-01-05,1,Vente,216000.0,10.0,A,RUE BASSE,0010,34670.0,34022,Baillargues,34,NaN,NaN,34022000AH0068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,19.0,1.0,S,sols,NaN,NaN,103.0,4.014809,43.660787
14,2023-173862,2023-01-04,1,Vente,82000.0,779.0,NaN,RUE DE SAINT HILAIRE,5115,34070.0,34172,Montpellier,34,NaN,NaN,34172000DR0558,NaN,NaN,2034.0,21.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,21.0,1.0,NaN,NaN,NaN,NaN,NaN,3.891022,43.590893
16,2023-173864,2023-01-06,1,Vente,76000.0,147.0,NaN,COUR DES CAMISARDS,0949,34080.0,34172,Montpellier,34,NaN,NaN,34172000MS0122,NaN,NaN,307.0,30.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,30.0,1.0,NaN,NaN,NaN,NaN,NaN,3.846555,43.619867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32389,2023-188408,2023-06-26,1,Vente,54150.0,2.0,NaN,RUE DES PENITENTS NOIRS,6250,34500.0,34032,Béziers,34,NaN,NaN,34032000PZ0272,NaN,NaN,20.0,30.56,21.0,NaN,22.0,NaN,NaN,NaN,NaN,NaN,3,2.0,Appartement,30.0,3.0,NaN,NaN,NaN,NaN,NaN,3.213315,43.345102
32390,2023-188409,2023-06-22,1,Vente,114000.0,2.0,NaN,RUE JEAN JACQUES ROUSSEAU,4440,34500.0,34032,Béziers,34,NaN,NaN,34032000LX0410,NaN,NaN,18.0,89.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,91.0,3.0,NaN,NaN,NaN,NaN,NaN,3.215119,43.340108
32405,2023-188418,2023-06-20,1,Vente,74000.0,14.0,NaN,AV GAMBETTA,3530,34500.0,34032,Béziers,34,NaN,NaN,34032000LX0764,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,50.0,2.0,NaN,NaN,NaN,NaN,NaN,3.216692,43.339397
32406,2023-188419,2023-06-19,1,Vente,180500.0,6.0,NaN,RUE DES OFFICIERS,1658,34300.0,34003,Agde,34,NaN,NaN,34003000OP0118,NaN,NaN,412.0,36.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,45.0,2.0,NaN,NaN,NaN,NaN,NaN,3.521735,43.284802


# Group Mutations

Same mutation (same mutation id) can have multiple properties. In this case, the mutation price is for the bundle. 

In [19]:
data_34_curated = data_34.groupby("id_mutation").agg({"valeur_fonciere": min, 
                                                      "longitude": min, 
                                                      "latitude": min,
                                                      "nombre_pieces_principales": sum, 
                                                      "surface_reelle_bati": sum,
                                                      "code_postal": min,
                                                      "nom_commune": min})

In [20]:
data_34_curated

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune
id_mutation,,,,,,,
2023-173855,158730.0,4.086656,43.558880,1.0,25.0,34280.0,La Grande-Motte
2023-173861,216000.0,4.014809,43.660787,3.0,43.0,34670.0,Baillargues
2023-173862,82000.0,3.891022,43.590893,1.0,21.0,34070.0,Montpellier
2023-173864,76000.0,3.846555,43.619867,1.0,30.0,34080.0,Montpellier
2023-173870,218500.0,3.886218,43.606132,3.0,70.0,34000.0,Montpellier
...,...,...,...,...,...,...,...
2023-188408,54150.0,3.213315,43.345102,3.0,30.0,34500.0,Béziers
2023-188409,114000.0,3.215119,43.340108,3.0,91.0,34500.0,Béziers
2023-188418,74000.0,3.216692,43.339397,2.0,50.0,34500.0,Béziers


# Analysis and clean

In [22]:
data_34_curated.isnull().describe()

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune
count,1214,1214,1214,1214,1214,1214,1214
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,1214,1214,1214,1214,1214,1214,1214


In [23]:
data_34_curated['valeur_mc'] = data_34_curated['valeur_fonciere'] / data_34_curated['surface_reelle_bati']

In [24]:
data_34_curated[["valeur_mc", "nombre_pieces_principales", "surface_reelle_bati"]].describe()

,valeur_mc,nombre_pieces_principales,surface_reelle_bati
count,1214.000000,1214.000000,1214.000000
mean,3241.259334,2.340198,48.621087
std,1612.637286,1.807856,39.184091
min,0.010526,0.000000,8.000000
25%,2106.870567,1.000000,25.000000
50%,3230.803030,2.000000,39.000000
75%,4166.666667,3.000000,60.000000
max,21041.666667,31.000000,591.000000


In [25]:
data_34_curated[data_34_curated["valeur_mc"] < 500]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-175591,7500.0,3.843600,43.635123,1.0,18.0,34080.0,Montpellier,416.666667
2023-176217,25000.0,3.692617,43.401730,3.0,59.0,34200.0,Sète,423.728814
2023-176458,1.0,3.700785,43.403695,4.0,95.0,34200.0,Sète,0.010526
2023-177653,500.0,3.703020,43.405361,3.0,61.0,34200.0,Sète,8.196721
2023-178340,6000.0,3.696870,43.401868,2.0,55.0,34200.0,Sète,109.090909
2023-180091,200000.0,3.798577,43.649172,22.0,520.0,34790.0,Grabels,384.615385
2023-181439,500.0,3.869755,43.608690,4.0,163.0,34000.0,Montpellier,3.067485
2023-182571,40000.0,3.170291,43.688841,4.0,81.0,34260.0,Le Bousquet-d'Orb,493.827160
2023-183470,16000.0,3.470235,43.312322,1.0,34.0,34300.0,Agde,470.588235


In [26]:
data_34_curated[data_34_curated["valeur_mc"] > 8000]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-178121,252500.0,4.135031,43.676172,0.0,12.0,34400.0,Lunel,21041.666667
2023-178689,210000.0,3.976282,43.543804,2.0,25.0,34130.0,Mauguio,8400.000000
2023-178724,330000.0,4.092313,43.557566,1.0,40.0,34280.0,La Grande-Motte,8250.000000
2023-180314,242000.0,4.080460,43.561437,1.0,28.0,34280.0,La Grande-Motte,8642.857143
2023-180853,485000.0,3.953522,43.534952,3.0,57.0,34250.0,Palavas-les-Flots,8508.771930
2023-181019,103000.0,3.881275,43.617552,1.0,11.0,34090.0,Montpellier,9363.636364
2023-181567,227840.0,3.870236,43.609357,1.0,26.0,34000.0,Montpellier,8763.076923
2023-183258,142120.0,3.870236,43.609357,1.0,16.0,34000.0,Montpellier,8882.500000
2023-183756,233870.0,3.529332,43.294384,1.0,18.0,34300.0,Agde,12992.777778


In [27]:
data_34_curated[data_34_curated["surface_reelle_bati"] < 10]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-178104,27000.0,3.872398,43.607581,1.0,9.0,34000.0,Montpellier,3000.0
2023-179125,63000.0,3.878096,43.611537,1.0,9.0,34000.0,Montpellier,7000.0
2023-181296,30000.0,3.881962,43.606934,1.0,8.0,34000.0,Montpellier,3750.0
2023-182263,30000.0,3.883175,43.605015,1.0,8.0,34070.0,Montpellier,3750.0


In [28]:
data_34_curated[data_34_curated["surface_reelle_bati"] > 200]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-175743,159000.0,3.322897,43.730584,11.0,230.0,34700.0,Lodève,691.304348
2023-177366,515000.0,3.762101,43.581887,9.0,209.0,34570.0,Pignan,2464.114833
2023-179675,242000.0,3.431210,43.626356,12.0,240.0,34800.0,Clermont-l'Hérault,1008.333333
2023-180091,200000.0,3.798577,43.649172,22.0,520.0,34790.0,Grabels,384.615385
2023-180492,1650000.0,3.694116,43.403133,31.0,591.0,34200.0,Sète,2791.878173
2023-181594,300000.0,3.857112,43.604822,9.0,286.0,34070.0,Montpellier,1048.951049
2023-183597,305000.0,3.212049,43.342585,14.0,283.0,34500.0,Béziers,1077.738516
2023-186436,381000.0,3.422766,43.461032,12.0,248.0,34120.0,Pézenas,1536.290323
2023-186903,191000.0,3.220343,43.339977,7.0,212.0,34500.0,Béziers,900.943396


In [29]:
data_34_curated[data_34_curated["nombre_pieces_principales"] > 10]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-175743,159000.0,3.322897,43.730584,11.0,230.0,34700.0,Lodève,691.304348
2023-179675,242000.0,3.431210,43.626356,12.0,240.0,34800.0,Clermont-l'Hérault,1008.333333
2023-180091,200000.0,3.798577,43.649172,22.0,520.0,34790.0,Grabels,384.615385
2023-180492,1650000.0,3.694116,43.403133,31.0,591.0,34200.0,Sète,2791.878173
2023-183597,305000.0,3.212049,43.342585,14.0,283.0,34500.0,Béziers,1077.738516
2023-186436,381000.0,3.422766,43.461032,12.0,248.0,34120.0,Pézenas,1536.290323


In [30]:
data_34_curated[data_34_curated["nombre_pieces_principales"] == 0]

,valeur_fonciere,longitude,latitude,nombre_pieces_principales,surface_reelle_bati,code_postal,nom_commune,valeur_mc
id_mutation,,,,,,,,
2023-178002,91000.0,3.925891,43.636461,0.0,22.0,34170.0,Castelnau-le-Lez,4136.363636
2023-178121,252500.0,4.135031,43.676172,0.0,12.0,34400.0,Lunel,21041.666667


In [31]:
filter_mask = (data_34_curated["valeur_mc"] < 8000 ) & (data_34_curated["valeur_mc"] >= 500 ) \
& (data_34_curated["surface_reelle_bati"] < 200 ) & (data_34_curated["surface_reelle_bati"] >= 10 ) \
& (data_34_curated["nombre_pieces_principales"] <= 10 ) & (data_34_curated["nombre_pieces_principales"] > 0 )

In [32]:
filter_mask.value_counts()

True     1178
False      36
dtype: int64

In [33]:
data_34_curated = data_34_curated[filter_mask]

In [34]:
data_34_curated[["valeur_mc", "nombre_pieces_principales", "surface_reelle_bati"]].describe()

,valeur_mc,nombre_pieces_principales,surface_reelle_bati
count,1178.000000,1178.000000,1178.000000
mean,3229.152610,2.261460,46.692699
std,1415.763983,1.364519,29.596074
min,526.315789,1.000000,10.000000
25%,2148.782344,1.000000,25.000000
50%,3248.529412,2.000000,39.000000
75%,4165.972222,3.000000,59.000000
max,7722.222222,10.000000,199.000000


In [35]:
data_34_curated.groupby("nom_commune").agg({"code_postal": "count", 
                                            "valeur_mc": np.mean, 
                                            "nombre_pieces_principales": np.mean, 
                                            "surface_reelle_bati": np.mean}).sort_values("code_postal", ascending=False)



,code_postal,valeur_mc,nombre_pieces_principales,surface_reelle_bati
nom_commune,,,,
Montpellier,417,3568.165822,2.021583,42.062350
Agde,121,3208.065230,1.900826,37.520661
Sète,113,3062.991648,2.398230,47.274336
Béziers,88,1510.584234,3.261364,64.954545
La Grande-Motte,76,4618.302415,1.486842,31.907895
...,...,...,...,...
Nissan-lez-Enserune,1,1530.769231,3.000000,65.000000
Olonzac,1,890.000000,3.000000,100.000000
Paulhan,1,842.758621,5.000000,145.000000


# Save curated data

In [36]:
CURATED_DATA_34_PATH = "data/curated_34.csv"

data_34_curated.to_csv(CURATED_DATA_34_PATH, index=None)